In [1]:
import os
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Activation, Flatten
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
import numpy as np

Using TensorFlow backend.


In [2]:
import os
import pickle

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [3]:
from keras.models import load_model


W projekcie będe wykorzystywał sieci konwolucyjne. Są one bardzo popularne przy rozwiązywaniu problemu klasyfikacji obrazów. Jest to odmiana sieci neuronowych w części warsw znajdują sie filtry, są to macierze, które w pewien sposób przemnaża sie przez wejściowe obrazki (operacja konwolucji) i dostajemy przez to kolejne wyściowe macierze (ich liczba jest równa liczbie filtrów). Filtry są obiektami, które będą uczone podczas procesu nauczania. W przypadku rozpoznawania obrazów ta taktyka może być skuteczna przez to, że sieci uczą się cech, które są niezmienicze ze wględu na przesunięcie o wektor. 

Po dostaniu inputu z Filtrów możemy zastosować na dla każdego elementu z macierzy zastosować funkcje aktywacji np. relu, funkcje sigmoidalną.

W sieciach konwolucyjnych wykozystuje się także warstwy, które transformują macierze na mniejsze. Jest tak zwana to warstwa pool.

Moj pierwszy Model słada się z dwóch warstw konwolucji.
Każda wejściowa macierz dla, której liczymy konwolucje dostaje "ramke" złożoną z zer w taki sposób aby 
podczas operacji konwolucji otrzymane macierze miała takie same wymiary co macierz wejściowa

Po każdej z nich stosuje funkcje aktywacyjną ReLu a następnie Zmiejszam wymiary macierzy za pomocą funkcji MaxPool2D. Funkcja ta dla podziału wejściowej macierzy na macierze n na m zwraca ich maximum. W moim modelu ustawiłem m=n=2

Przy każdej iteracji algorytmu uczącego stosuje dropout, oznacza to, że część wyjść z danej warstwy zostanie wyzerowana z pewnym prawdopodobieństwem. Ma to zmiejszyć możliwe zoverfitowanie sieci do danych oraz zapobiec temu, że tylko część neuronów będzie brała aktywny udział w predykcjach sieci. Dla warstw konwolucyjnych w tym modelu ustanowiłem równe 0.23

Wyjście z tych dwóch warstw konwolucji zostanie zamienione na wektor i potem przepuszczone przez zwykłą sieć neuronową z jedną ukrytą warstwą. Dla niej dropout wynosi 0.5.

Jako funkcji kosztu używam crossetropy a jako algorytmu szukania jej miniumum metode "ADAM"

Hiperparametry dla tego Modelu

conv_layer1_size odpowiada za to jaka jest liczba filtrów w pierwszej warstwie konwolucji

conv_layer2_size odpowiada za to jaka jest liczba filtrów w drugiej warstwie konwolucji

layer3_size odpowiada za liczbę neuronów trzeciej warstwie jest to już warstwa nie konwolucji

kernel_size to wymiar filtru dla obu warst konwolucji. Dokładniej macierz tworząca dany filtr będzie rozmiaru (kernel_size,kernel_size)

Reszta parametrów tej sieci konwolucyjnych będzie ustalona

Testowanie Modelu

Bedę uczył sieci na pierwszych 40000 przykładach ze zbioru treningowego. Natomiast zbiorem walidującym będzie ostanie 10000 przykładów ze zbioru uczącego. 

Będe starał się wyprodukować modele za pomocą funkcji model_generator je do tablicy, po ich fitowaniu na części danych będe, usuwał modele, które mają najmniejsze accurasy na zbiorze validującym lub jego podzbiorze. Gdy zostanie mi tylko jeden model będe go fittował do czasu aż jego accurasy na na zbiorze testowym zacznie spadać a potem wyśle jego predykcje

In [11]:
def save_labels(arr, filename):
    pd_array = pd.DataFrame(arr)
    pd_array.index.names = ["Id"]
    pd_array.columns = ["Prediction"]
    pd_array.to_csv(filename)

def load_labels(filename):
    return pd.read_csv(filename, index_col=0).values.ravel()

In [12]:
X_train = np.load("X_train.npy")
y_train = load_labels("y_train.csv")
X_test = np.load("X_test.npy")
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')
X_train /= np.max(X_train) 
X_test /= np.max(X_test) 
X_train=X_train.reshape(-1,32,32,3)
Y_train = np_utils.to_categorical(y_train, 10) 

In [66]:
X_test=X_test.reshape(-1,32,32,3)

In [13]:
#generujemy zadany model
def model_generator(conv_layer1_size,conv_layer2_size,layer3_size,kernel_size):
    drop_prob_1=0.23
    drop_prob_2=0.5
    pool_size=2
    inp = Input(shape=(32, 32,3))
    conv_1 = Convolution2D(conv_layer1_size, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
    pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_1)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    conv_2 = Convolution2D(conv_layer2_size, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
    pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    flat = Flatten()(drop_2)
    hidden = Dense(layer3_size, activation='relu')(flat)
    drop_3 = Dropout(drop_prob_2)(hidden)
    out = Dense(10, activation='softmax')(drop_3)
    model = Model(inputs=inp, outputs=out)
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']) 
    return model

In [14]:
#trenujemy modele w tablicy i zwracamy ich wynik dla zbioru walidującego
def models_scory(Model_table,X_train,y_train,X_ewal,y_ewal,nazwa_pliku,epochs=10):
    i=0
    for model in Model_table:
        i=i+1       
        model.fit(X_train,y_train,batch_size=32, epochs=epochs)
        model.save(nazwa_pliku+str(i)+'.h5') #zapisuje model
    return np.array([(model,model.evaluate(X_ewal,y_ewal, verbose=1)[1]) for model in Model_table]) 
#zwracamy pary (model,accuracy na zbiorze validującym)  

In [19]:

def load_model_tab(nazwa_pliku,n):       
     return [load_model(nazwa_pliku+str(i+1)+".h5") for i in range(n)]

In [35]:
# usuwamy połowe modeli, które mają najgorsze accuracy
#gdy liczba modeli jest nieparzysta to zostaje o jeden więcej
def delete_bad_model(Model_score_table):
    sort_tab=sorted(Model_score_table, key=lambda Model_score_table: Model_score_table[1]) 
    sort_tab= sort_tab[int(len(sort_tab)/2):len(sort_tab)]
    return [x[0] for x in sort_tab]

In [126]:
#Generujemy 20 lowych modeli o różnych hiperparametrach
Rand_model_table=[model_generator(
    np.random.randint(20,71), #liczba filtrów w pierszej warstwie konwolucji 
    np.random.randint(20,71), #liczba filtrów w drugiej warstwie konwolucji
    np.random.randint(100,500), #liczba filtrów w trzeciej warstwie (zwykła warstwa)
    np.random.randint(2,4), #rozmiar filtrów dla obu warst konwolucji 
)   for i in range(20)]

In [127]:
#uczenie modeli i usuwanie zlych modeli
tab=models_scory(Rand_model_table,X_train[0:5000],Y_train[0:5000],X_train[49000:50000],Y_train[49000:50000],"rand1.",epochs=5)

Epoch 1/5
5000/5000 [==============================] - 12s - loss: 2.2041 - acc: 0.1732    
Epoch 2/5
5000/5000 [==============================] - 10s - loss: 1.9406 - acc: 0.2860    - ETA: 0s - loss: 1.9405 - acc: 0.286
Epoch 3/5
5000/5000 [==============================] - 11s - loss: 1.8273 - acc: 0.3282    
Epoch 4/5
5000/5000 [==============================] - 11s - loss: 1.7379 - acc: 0.3794    
Epoch 5/5
5000/5000 [==============================] - 11s - loss: 1.6853 - acc: 0.3866    
Epoch 1/5
5000/5000 [==============================] - 17s - loss: 2.1897 - acc: 0.1872    
Epoch 2/5
5000/5000 [==============================] - 15s - loss: 1.9597 - acc: 0.2854    
Epoch 3/5
5000/5000 [==============================] - 15s - loss: 1.8224 - acc: 0.3234    
Epoch 4/5
5000/5000 [==============================] - 15s - loss: 1.7571 - acc: 0.3530    
Epoch 5/5
5000/5000 [==============================] - 15s - loss: 1.6977 - acc: 0.3758    
Epoch 1/5
5000/5000 [======================

In [129]:
tab

array([[<keras.engine.training.Model object at 0x000001CD3C822A58>,
        0.45500000000000002],
        0.41199999999999998],
        0.48899999999999999],
        0.46300000000000002],
        0.45100000000000001],
        0.45800000000000002],
        0.44400000000000001],
       [<keras.engine.training.Model object at 0x000001CD462AAEF0>, 0.44],
        0.47999999999999998],
        0.46000000000000002],
        0.45700000000000002],
        0.45300000000000001],
        0.46600000000000003],
        0.48199999999999998],
       [<keras.engine.training.Model object at 0x000001CD47435320>, 0.505],
        0.48699999999999999],
        0.44400000000000001],
        0.48499999999999999],
        0.46600000000000003],
        0.46500000000000002]], dtype=object)

In [132]:
tab=models_scory(Rand_model_table,X_train[0:15000],Y_train[0:15000],X_train[45000:50000],Y_train[45000:50000],"rand2.",epochs=5)

Epoch 1/5
15000/15000 [==============================] - 32s - loss: 1.6588 - acc: 0.4026    
Epoch 2/5
15000/15000 [==============================] - 31s - loss: 1.5805 - acc: 0.4331    
Epoch 3/5
15000/15000 [==============================] - 32s - loss: 1.5273 - acc: 0.4525    
Epoch 4/5
15000/15000 [==============================] - 30s - loss: 1.4810 - acc: 0.4695    
Epoch 5/5
15000/15000 [==============================] - 30s - loss: 1.4314 - acc: 0.4831    
Epoch 1/5
15000/15000 [==============================] - 48s - loss: 1.6763 - acc: 0.3906    
Epoch 2/5
15000/15000 [==============================] - 45s - loss: 1.5986 - acc: 0.4269    
Epoch 3/5
15000/15000 [==============================] - 46s - loss: 1.5366 - acc: 0.4425    
Epoch 4/5
15000/15000 [==============================] - 44s - loss: 1.4904 - acc: 0.4702    
Epoch 5/5
15000/15000 [==============================] - 44s - loss: 1.4440 - acc: 0.4858    
Epoch 1/5
15000/15000 [==============================] - 55s

In [133]:
tab

array([[<keras.engine.training.Model object at 0x000001CD3C822A58>,
        0.52800000000000002],
        0.53680000000000005],
        0.57120000000000004],
        0.57140000000000002],
        0.55600000000000005],
       [<keras.engine.training.Model object at 0x000001CD45DC4CF8>, 0.5534],
        0.54959999999999998],
       [<keras.engine.training.Model object at 0x000001CD462AAEF0>, 0.5222],
        0.56499999999999995],
        0.53159999999999996],
        0.53400000000000003],
        0.54239999999999999],
        0.55520000000000003],
        0.58660000000000001],
        0.58520000000000005],
        0.57840000000000003],
        0.55279999999999996],
        0.57940000000000003],
        0.57540000000000002],
        0.56659999999999999]], dtype=object)

In [134]:
Rand_model_table2=delete_bad_model(tab)

In [136]:
#Zostało 10 modeli
tab2=models_scory(Rand_model_table2,X_train[15000:20000],Y_train[15000:20000],X_train[45000:50000],Y_train[45000:50000],"rand3.",epochs=5)

Epoch 1/5
5000/5000 [==============================] - 23s - loss: 1.4322 - acc: 0.4912    
Epoch 2/5
5000/5000 [==============================] - 23s - loss: 1.3384 - acc: 0.5312    
Epoch 3/5
5000/5000 [==============================] - 21s - loss: 1.2863 - acc: 0.5420    
Epoch 4/5
5000/5000 [==============================] - 26s - loss: 1.2345 - acc: 0.5602    
Epoch 5/5
5000/5000 [==============================] - 23s - loss: 1.1708 - acc: 0.5852    
Epoch 1/5
5000/5000 [==============================] - 28s - loss: 1.4279 - acc: 0.4954    
Epoch 2/5
5000/5000 [==============================] - 28s - loss: 1.3167 - acc: 0.5428    
Epoch 3/5
5000/5000 [==============================] - 27s - loss: 1.2653 - acc: 0.5566    
Epoch 4/5
5000/5000 [==============================] - 28s - loss: 1.1661 - acc: 0.5904    
Epoch 5/5
5000/5000 [==============================] - 27s - loss: 1.0952 - acc: 0.6088    
Epoch 1/5
5000/5000 [==============================] - 23s - loss: 1.3647 - acc:

In [22]:
Rand_model_table2=load_model_tab("rand3.",10)

In [23]:
tab2=models_scory(Rand_model_table2,X_train[20000:25000],Y_train[20000:25000],X_train[45000:50000],Y_train[45000:50000],"rand4.",epochs=5)

Epoch 1/5
5000/5000 [==============================] - 23s - loss: 1.0982 - acc: 0.6068    
Epoch 2/5
5000/5000 [==============================] - 23s - loss: 1.0362 - acc: 0.6264    
Epoch 3/5
5000/5000 [==============================] - 24s - loss: 0.9800 - acc: 0.6524    
Epoch 4/5
5000/5000 [==============================] - 23s - loss: 0.9361 - acc: 0.6698    
Epoch 5/5
5000/5000 [==============================] - 23s - loss: 0.8876 - acc: 0.6798    
Epoch 1/5
5000/5000 [==============================] - 29s - loss: 1.3438 - acc: 0.5194    
Epoch 2/5
5000/5000 [==============================] - 30s - loss: 1.2561 - acc: 0.5556    
Epoch 3/5
5000/5000 [==============================] - 29s - loss: 1.1777 - acc: 0.5732    
Epoch 4/5
5000/5000 [==============================] - 27s - loss: 1.1014 - acc: 0.6106    
Epoch 5/5
5000/5000 [==============================] - 28s - loss: 1.0383 - acc: 0.6290    
Epoch 1/5
5000/5000 [==============================] - 23s - loss: 1.3418 - acc:

In [24]:
tab2

array([[<keras.engine.training.Model object at 0x000002A741B728D0>,
        0.60219999999999996],
        0.58899999999999997],
       [<keras.engine.training.Model object at 0x000002A744A3BC88>, 0.5978],
        0.61140000000000005],
        0.59640000000000004],
       [<keras.engine.training.Model object at 0x000002A74AD9EAC8>, 0.6028],
        0.60419999999999996],
        0.61560000000000004],
        0.61499999999999999],
        0.58819999999999995]], dtype=object)

In [28]:
#zostało 5 modeli
Rand_model_table3=delete_bad_model(tab2)


In [30]:
tab3=models_scory(Rand_model_table3,X_train[0:30000],Y_train[0:30000],X_train[45000:50000],Y_train[45000:50000],"rand5.",epochs=5)

Epoch 1/5
30000/30000 [==============================] - 116s - loss: 1.1612 - acc: 0.5891   
Epoch 2/5
30000/30000 [==============================] - 114s - loss: 1.0996 - acc: 0.6134   
Epoch 3/5
30000/30000 [==============================] - 116s - loss: 1.0672 - acc: 0.6228   
Epoch 4/5
30000/30000 [==============================] - 117s - loss: 1.0226 - acc: 0.6391   
Epoch 5/5
30000/30000 [==============================] - 116s - loss: 0.9914 - acc: 0.6491   
Epoch 1/5
30000/30000 [==============================] - 118s - loss: 1.1358 - acc: 0.6016   
Epoch 2/5
30000/30000 [==============================] - 119s - loss: 1.0775 - acc: 0.6180   
Epoch 3/5
30000/30000 [==============================] - 118s - loss: 1.0355 - acc: 0.6330   
Epoch 4/5
30000/30000 [==============================] - 123s - loss: 1.0072 - acc: 0.6439   
Epoch 5/5
30000/30000 [==============================] - 126s - loss: 0.9602 - acc: 0.6595   
Epoch 1/5
30000/30000 [==============================] - 109

In [31]:
tab3

array([[<keras.engine.training.Model object at 0x000002A74AD9EAC8>, 0.6482],
        0.65480000000000005],
        0.65539999999999998],
        0.63580000000000003],
        0.65380000000000005]], dtype=object)

In [37]:
#zostały 3 modele
Rand_model_table4=delete_bad_model(tab3)

In [38]:
tab4=models_scory(Rand_model_table4,X_train[0:40000],Y_train[0:40000],X_train[45000:50000],Y_train[45000:50000],"rand5.",epochs=7)

Epoch 1/7
40000/40000 [==============================] - 130s - loss: 1.0515 - acc: 0.6313   - ETA
Epoch 2/7
40000/40000 [==============================] - 129s - loss: 1.0211 - acc: 0.6400   
Epoch 3/7
40000/40000 [==============================] - 130s - loss: 0.9983 - acc: 0.6464   
Epoch 4/7
40000/40000 [==============================] - 130s - loss: 0.9883 - acc: 0.6523   
Epoch 5/7
40000/40000 [==============================] - 133s - loss: 0.9557 - acc: 0.6627   
Epoch 6/7
40000/40000 [==============================] - 138s - loss: 0.9457 - acc: 0.6667   
Epoch 7/7
40000/40000 [==============================] - 141s - loss: 0.9164 - acc: 0.6731   
Epoch 1/7
40000/40000 [==============================] - 163s - loss: 1.0299 - acc: 0.6418   
Epoch 2/7
40000/40000 [==============================] - 182s - loss: 0.9930 - acc: 0.6515   
Epoch 3/7
40000/40000 [==============================] - 174s - loss: 0.9653 - acc: 0.6614   
Epoch 4/7
40000/40000 [==============================] 

In [40]:
tab5=models_scory(Rand_model_table4,X_train[0:40000],Y_train[0:40000],X_train[45000:50000],Y_train[45000:50000],"rand6.",epochs=10)

Epoch 1/10
40000/40000 [==============================] - 146s - loss: 0.9005 - acc: 0.6805   
Epoch 2/10
40000/40000 [==============================] - 141s - loss: 0.8878 - acc: 0.6839   
Epoch 3/10
40000/40000 [==============================] - 144s - loss: 0.8725 - acc: 0.6909   
Epoch 4/10
40000/40000 [==============================] - 146s - loss: 0.8618 - acc: 0.6910   
Epoch 5/10
40000/40000 [==============================] - 147s - loss: 0.8450 - acc: 0.6987   
Epoch 6/10
40000/40000 [==============================] - 144s - loss: 0.8350 - acc: 0.6998   
Epoch 7/10
40000/40000 [==============================] - 144s - loss: 0.8168 - acc: 0.7075   
Epoch 8/10
40000/40000 [==============================] - 146s - loss: 0.8107 - acc: 0.7089   
Epoch 9/10
40000/40000 [==============================] - 143s - loss: 0.7972 - acc: 0.7130   
Epoch 10/10
40000/40000 [==============================] - 134s - loss: 0.7935 - acc: 0.7146   
Epoch 1/10
40000/40000 [=========================

In [44]:
#zostały 2 modele
Rand_model_table5=delete_bad_model(tab5)

In [45]:
tab5=models_scory(Rand_model_table5,X_train[0:40000],Y_train[0:40000],X_train[40000:50000],Y_train[40000:50000],"rand7.",epochs=10)

Epoch 1/10
40000/40000 [==============================] - 178s - loss: 0.7147 - acc: 0.7444   
Epoch 2/10
40000/40000 [==============================] - 170s - loss: 0.7028 - acc: 0.7499   
Epoch 3/10
40000/40000 [==============================] - 161s - loss: 0.6846 - acc: 0.7564   
Epoch 4/10
40000/40000 [==============================] - 156s - loss: 0.6776 - acc: 0.7579   
Epoch 5/10
40000/40000 [==============================] - 160s - loss: 0.6686 - acc: 0.7602   
Epoch 6/10
40000/40000 [==============================] - 165s - loss: 0.6686 - acc: 0.7601   
Epoch 7/10
40000/40000 [==============================] - 159s - loss: 0.6639 - acc: 0.7645   
Epoch 8/10
40000/40000 [==============================] - 167s - loss: 0.6575 - acc: 0.7642   
Epoch 9/10
40000/40000 [==============================] - 165s - loss: 0.6431 - acc: 0.7727   
Epoch 10/10
40000/40000 [==============================] - 157s - loss: 0.6299 - acc: 0.7744   
Epoch 1/10
40000/40000 [=========================

In [47]:
#został ostatni model fittuje go dopuki accuracy na zbiorze validującym nie będzie spadać
#zaczynamy monitotować accuracy podczas uczenia ostatniego modelu jeśli spadnie to przerywamy fittowanie
Rand_model_table5=delete_bad_model(tab5)

In [48]:
tab5=models_scory(Rand_model_table5,X_train[0:40000],Y_train[0:40000],X_train[40000:50000],Y_train[40000:50000],"ostatni.",epochs=1)

Epoch 1/1
 9984/10000 [============================>.] - ETA: 0s

In [49]:

tab5

array([[<keras.engine.training.Model object at 0x000002A748844E10>,
        0.68289999999999995]], dtype=object)

In [50]:
tab5=models_scory(Rand_model_table5,X_train[0:40000],Y_train[0:40000],X_train[40000:50000],Y_train[40000:50000],"ostatni.",epochs=1)

Epoch 1/1
 9952/10000 [============================>.] - ETA: 0s

In [51]:
tab5

array([[<keras.engine.training.Model object at 0x000002A748844E10>,
        0.68310000000000004]], dtype=object)

In [52]:
tab5=models_scory(Rand_model_table5,X_train[0:40000],Y_train[0:40000],X_train[40000:50000],Y_train[40000:50000],"ostatni_2.",epochs=10)

Epoch 1/10
40000/40000 [==============================] - 133s - loss: 0.6178 - acc: 0.7798   
Epoch 2/10
40000/40000 [==============================] - 139s - loss: 0.6184 - acc: 0.7768   
Epoch 3/10
40000/40000 [==============================] - 137s - loss: 0.6043 - acc: 0.7824   
Epoch 4/10
40000/40000 [==============================] - 134s - loss: 0.6043 - acc: 0.7830   
Epoch 5/10
40000/40000 [==============================] - 135s - loss: 0.5962 - acc: 0.7845   
Epoch 6/10
40000/40000 [==============================] - 134s - loss: 0.5864 - acc: 0.7897   
Epoch 7/10
40000/40000 [==============================] - 138s - loss: 0.5892 - acc: 0.7903   
Epoch 8/10
40000/40000 [==============================] - 138s - loss: 0.5803 - acc: 0.7915   
Epoch 9/10
40000/40000 [==============================] - 139s - loss: 0.5774 - acc: 0.7924   
Epoch 10/10
 9984/10000 [============================>.] - ETA: 0s

In [54]:
#accurasy s
tab5

array([[<keras.engine.training.Model object at 0x000002A748844E10>,
        0.68120000000000003]], dtype=object)

In [ ]:
#Accurasy modelu na danych validujących spadło więc model zaczyna się overfittować

In [82]:
#Predykcja danych testowych
Y_test_predicted=model_ostateczzny.predict(X_test)

In [106]:
Y_test_predicted.shape

(10000, 10)

In [107]:
y_test_predict=np.array([np.argmax(x) for x in Y_test_predicted])

In [110]:
save_labels(y_test_predict, "y_pred.csv")